In [ ]:
!pip install wget
import wget
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

# Daten laden und präparieren

In [ ]:
if not os.path.isdir("data/PetImages"):
    wget.download("https://oshi.at/twJYYz", bar=wget.bar_adaptive)

    import zipfile
    with zipfile.ZipFile("JuIO.zip", "r") as zip_ref:
        zip_ref.extractall("")


train_path = 'data/PetImages/train'
valid_path = 'data/PetImages/valid'
test_path = 'data/PetImages/test'

In [ ]:
image_size = 224
batch_size = 10

classes = ['Cat', 'Dog']

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path,
                         target_size=(image_size, image_size),
                         classes=classes,
                         batch_size=batch_size)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path,
                         target_size=(image_size, image_size),
                         classes=classes,
                         batch_size=batch_size)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path,
                         target_size=(image_size, image_size),
                         classes=classes,
                         batch_size=batch_size, shuffle=False)

# Neuronales Netz erstellen

## Transfer Learning Definition
Wir betrachten ein _unterschiedliches_, aber ähnliches Problem. Wenn hierzu bereits ein neuronales Netz besteht, welches dieses gut löst, können wir das bereits bestehende neuronale Netz verwenden und leicht abändern und ein paar Ebenen des Netzes neu trainieren, um es auf unser Problem zu spezifizieren.

TensorFlow bietet in seiner Bibliothek bereits viele verschiedene vortrainierte Netze an. 
Diese kann man sich unter [https://www.tensorflow.org/api_docs/python/tf/keras/applications?hl=de](https://www.tensorflow.org/api_docs/python/tf/keras/applications?hl=de) ansehen.

Wir verwenden nun das VGG16 Modell für Transfer Learning.
Unser Ziel ist es, ein neuronales Netz zu erstellen, welches Bilder von Katzen und Hunden unterscheiden kann.
Wie gerade gesehen, funktioniert dies mit einfachen CNNs nicht so gut.
Deshalb versuchen wir es mit Transfer Learning bei einem bereits trainierten neuronalen Netz,
welches wir auf unsere Bedürfnisse spezialisieren.

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()
vgg16_model.summary()

Wir erstellen nun ein eigenes Modell, welches die gleichen Ebenen wie VGG16 hat außer der letzten.

In [ ]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

Da wir die Ebenen vor unserer Spezialisierung (dem letzten voll vernetzen neuronalen Netz) nicht erneut trainieren wollen,
werden wir diese freezen, d. h. `trainable = False` setzen.

Statt dem Dense Layer mit 1000 Outputs verwenden wir ein Dense Layer mit 2 Outputs - Katze oder Hund.

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.add(Dense(units=2, activation='softmax'))
model.summary()

# Neuronales Netz trainieren

Nun haben wir ein Modell, welches nur 8194 anpassbare Parameter hat statt 134 Millionen.
Wir verwenden wieder `Adam` als Optimizer und categorical crossentropy loss als loss funktion.

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=3, verbose=2)

# Vorhersagen auf Testdatensatz

Nun betrachten wir wieder das Test-Set.

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_cm(y_true, y_pred, classes, title='Konfusionsmatrix', cmap=plt.cm.Blues):
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() * 2. / 3
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Tatsächlich')
    plt.xlabel('Vorhersage')

In [ ]:
predictions = model.predict(x=test_batches, verbose=2)

In [ ]:
import numpy as np
preds = np.argmax(predictions, axis=1)
print("Predictions:\n", preds)
print("Actual:\n", test_batches.classes)
print("Accuracy:", sum(preds == test_batches.classes)/preds.shape[0])

In [ ]:
plot_cm(test_batches.classes, preds, classes)